In [350]:
import pandas as pd
from datetime import datetime, timedelta
import random
import matplotlib.pyplot as plt

from weasyprint import HTML

import plotly.graph_objs as go
import plotly.io as pio
import base64

In [351]:
data = pd.read_csv("data/example_data.csv")

# data_copy = data.copy()

data = data[data["event_type"] == "enter"]
data["detection_time"] = pd.to_datetime(data["detection_time"])


data["day_of_week"] = data["detection_time"].dt.day_name()
data["week_of_month"] = data["detection_time"].dt.day // 7 + 1
data["month_of_year"] = data["detection_time"].dt.month
data['hour_of_day'] = data['detection_time'].dt.hour
data['quarter'] = data['detection_time'].dt.quarter



days_translation = {
    "Monday": "Pazartesi",
    "Tuesday": "Salı",
    "Wednesday": "Çarşamba",
    "Thursday": "Perşembe",
    "Friday": "Cuma",
    "Saturday": "Cumartesi",
    "Sunday": "Pazar",
}
data["day_of_week"] = data["day_of_week"].map(days_translation)

gender_name = {
    0: "male",
    1: "female"
}

data["gender"] = data["gender"].map(gender_name)
data.head()

,id,camera_id,detection_time,label,confidence,event_time,event_type,gender,gender_confidence,age_range,age_confidence,day_of_week,week_of_month,month_of_year,hour_of_day,quarter
0,1,1,2024-09-03 13:33:29,person counter,0.84,2024-09-03T13:33:29,enter,female,0.75,30-40,0.82,Salı,1,9,13,3
1,2,1,2024-03-11 13:42:54,person counter,0.76,2024-03-11T13:42:54,enter,female,0.69,20-25,0.83,Pazartesi,2,3,13,1
6,7,1,2024-04-22 09:05:14,person counter,0.81,2024-04-22T09:05:14,enter,female,0.59,30-40,0.87,Pazartesi,4,4,9,2
7,8,1,2024-11-25 01:37:38,person counter,0.86,2024-11-25T01:37:38,enter,male,0.51,20-25,0.93,Pazartesi,4,11,1,4
8,9,1,2024-02-15 07:07:54,person counter,0.99,2024-02-15T07:07:54,enter,female,0.91,30-40,0.81,Perşembe,3,2,7,1


In [352]:
def image_to_ascii(fig):
    img_bytes = pio.to_image(fig, format="png")
    return base64.b64encode(img_bytes).decode("ascii")

def daily_chart(data):
    days_of_week_ordered = ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"]
    daily_data = data["day_of_week"].value_counts().reindex(days_of_week_ordered)

    
    data1 = go.Bar(x=daily_data.index, y=daily_data)
    fig = go.Figure(data=data1)
    fig.update_layout(
        margin=dict(l=70, r=30, t=45, b=0),
        xaxis_tickangle=-45,
        title={"text": "Aylık Cinsiyet Dağılımı", "x": 0.5},
        title_font_size=30, 
        xaxis_tickfont_size=18,  
    )  # template="simple_white"
    return fig

def weekly_chart(data):
    daily_data = data["day_of_week"].value_counts()
    x_labels = [f"{i}. Hafta" for i in range(1, 6)]
    data1 = go.Bar(x=x_labels, y=daily_data)
    fig = go.Figure(data=data1)
    fig.update_layout(
        margin=dict(l=30, r=70, t=45, b=0),
        xaxis_tickangle=-45,
        title={"text": "Haftalara Göre Ziyaretçi Sayısı", "x": 0.5},
        title_font_size=30, 
        xaxis_tickfont_size=18,  
    )  # template="simple_white"
    return fig

def monthly_chart(data):
    monthly_data = data["month_of_year"].value_counts()
    month_names = ["Ocak", "Subat", "Mart", "Nisan", "Mayis", "Haziran", "Temmuz", "Agustos", "Eylül", "Ekim", "Kasim", "Aralik"]

    data1 = go.Bar(x=month_names, y=monthly_data)
    fig = go.Figure(data=data1)

    fig.update_layout(
        width=700,
        height=300,
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis_tickangle=-45,
        title={"text": "Aylara Göre Ziyaretçi Sayısı", "x": 0.5},
    )  # template="simple_white"

    return fig

def daily_chart_gender(data):
    days_of_week_ordered = ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"]

    daily_data = data.groupby(['day_of_week', 'gender']).size().unstack()
    daily_data = daily_data.reindex(days_of_week_ordered)
    
    fig = go.Figure()
    for gender in daily_data.columns:
        fig.add_trace(go.Bar(x=daily_data.index, y=daily_data[gender], name=gender))

    fig.update_layout(
        barmode='group',
        margin=dict(l=70, r=30, t=45, b=0),
        xaxis_tickangle=-45,
        title={'text': 'Haftanın Günlerine Göre Cinsiyet Dağılımı', 'x': 0.5},

        legend=dict(
                    x=0.5, y=-0.40,
                    orientation="h",
                    xanchor="center",
                    yanchor="bottom"
                ),
        title_font_size=30, 
        xaxis_tickfont_size=18,  
        )

    return fig

def weekly_chart_gender(data):
    weekly_data = data.groupby(['day_of_week', 'gender']).size().unstack()
    x_labels = [f"{i}. Hafta" for i in range(1, 6)]

    fig = go.Figure()
    for gender in weekly_data.columns:
        fig.add_trace(go.Bar(x=x_labels, y=weekly_data[gender], name=gender))

    fig.update_layout(
        barmode='group',
        margin=dict(l=30, r=70, t=45, b=0),
        xaxis_tickangle=-45,
        title={'text': 'Haftalık Cinsiyet Dağılımı', 'x': 0.5},

        legend=dict(
                    x=0.5, y=-0.40,
                    orientation="h",
                    xanchor="center",
                    yanchor="bottom"
                ),
        title_font_size=30, 
        xaxis_tickfont_size=18,  
        )

    return fig

def monthly_chart_gender(data):
    monthly_data = data.groupby(['month_of_year', 'gender']).size().unstack()
    month_names = ["Ocak", "Subat", "Mart", "Nisan", "Mayis", "Haziran", "Temmuz", "Agustos", "Eylül", "Ekim", "Kasim", "Aralik"]

    fig = go.Figure()
    for gender in monthly_data.columns:
        fig.add_trace(go.Bar(x=month_names, y=monthly_data[gender], name=gender))

    fig.update_layout(
        width=700,
        height=300,
        margin=dict(l=0, r=0, t=30, b=0),
        title={"text": "Aylık Cinsiyet Dağılımı", "x": 0.5},
        xaxis_tickangle=-45,
        legend=dict(
                x=0.5, y=-0.40,
                orientation="h",
                xanchor="center",
                yanchor="bottom"
            )
    )  # template="simple_white"


    return fig

def hourly_chart(data):
    hourly_data = data['hour_of_day'].value_counts().sort_index()
    hour_names = [f"{i:02d}:00" for i in range(1, 25)]

    fig = go.Figure(data=go.Bar(x=hour_names, y=hourly_data))

    fig.update_layout(
        title={'text': 'Saatlere Göre Dağılım', 'x': 0.5},
        width=700,
        height=300,
        xaxis_tickangle=-45,
        margin=dict(l=0, r=0, t=30, b=0),
    )
    return fig

def hourly_gender_chart(data):
    hourly_gender_data = data.groupby(['hour_of_day', 'gender']).size().unstack()
    hour_names = [f"{i:02d}:00" for i in range(1, 25)]

    fig = go.Figure()
    for gender in hourly_gender_data.columns:
        fig.add_trace(go.Bar(x=hour_names, y=hourly_gender_data[gender], name=gender))

    fig.update_layout(
        title={'text': 'Saatlere Göre Cinsiyet Dağılımı', 'x': 0.5},
        width=700,
        height=300,
        barmode='group',
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis_tickangle=-45,
        legend=dict(
                    x=0.5, y=-0.40,
                    orientation="h",
                    xanchor="center",
                    yanchor="bottom"
                )
    )

    return fig

def monthly_day_distribution_count(data):
    grouped_data = data.groupby(['month_of_year', 'day_of_week']).size().reset_index(name='count')
    month_names = ["Ocak", "Subat", "Mart", "Nisan", "Mayis", "Haziran", "Temmuz", "Agustos", "Eylül", "Ekim", "Kasim", "Aralik"]

    fig = go.Figure()
    for month in sorted(grouped_data['month_of_year'].unique()):
        month_data = grouped_data[grouped_data['month_of_year'] == month]
        fig.add_trace(go.Bar(x=month_data['day_of_week'], y=month_data['count'], name=month_names[month-1]))

    fig.update_layout(
        title={'text': 'Aylara Göre Günlerin Sayım Dağılımı', 'x': 0.5},
        width=700,
        height=300,
        barmode='group',
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis_tickangle=-45,
        # legend=dict(
        #             x=0.5, y=-0.40,
        #             orientation="h",
        #             xanchor="center",
        #             yanchor="bottom"
        #         )
    )

    return fig


def monthly_day_distribution_average(data):
    grouped_data = data.groupby(['month_of_year', 'day_of_week'])['confidence'].mean().reset_index()
    month_names = ["Ocak", "Subat", "Mart", "Nisan", "Mayis", "Haziran", "Temmuz", "Agustos", "Eylül", "Ekim", "Kasim", "Aralik"]

    fig = go.Figure()
    for month in sorted(grouped_data['month_of_year'].unique()):
        month_data = grouped_data[grouped_data['month_of_year'] == month]
        fig.add_trace(go.Bar(x=month_data['day_of_week'], y=month_data['confidence'], name=month_names[month-1]))

    fig.update_layout(
        title={'text': 'Aylara Göre Günlerin Ortalaması Dağılımı', 'x': 0.5},
        width=700,
        height=300,
        barmode='group',
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis_tickangle=-45,
        # legend=dict(
        #             x=0.5, y=-0.40,
        #             orientation="h",
        #             xanchor="center",
        #             yanchor="bottom"
        #         )
    )

    return fig


def quarter_pie_chart(data):
    quarter_counts = data['quarter'].value_counts()
    x_labels = ["Q1", "Q2", "Q3", "Q4"]

    fig = go.Figure(data=go.Pie(labels=x_labels, values=quarter_counts))

    fig.update_layout(
        title={'text': 'Yılın Çeyreklerine Göre Dağılım', 'x': 0.5},
        legend=dict(
                    x=0.5, y=-0.40,
                    orientation="h",
                    xanchor="center",
                    yanchor="bottom"
                )
    )

    return fig
quarter_pie_chart(data)


def gender_distribution_pie_chart(data):
    gender_counts = data['gender'].value_counts()
    fig = go.Figure(data=go.Pie(labels=gender_counts.index, values=gender_counts, hole=.3))
    fig.update_layout(
        title={'text': 'Cinsiyet Dağılımı', 'x': 0.5},
        legend=dict(
                    x=0.5, y=-0.40,
                    orientation="h",
                    xanchor="center",
                    yanchor="bottom"
                )

    )
    return fig



def heatmap_hour_day(data):
    days_of_week_ordered = ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"][::-1]

    grouped_data = data.groupby(['day_of_week', 'hour_of_day']).size().reset_index(name='count')
    heatmap_data = grouped_data.pivot(index='day_of_week', columns='hour_of_day', values='count')

    heatmap_data = heatmap_data.reindex(days_of_week_ordered)

    hour_names = [f"{i:02d}:00" for i in range(1, 25)]

    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data.values,
        x=hour_names,
        y=heatmap_data.index,
        colorscale='Viridis'
    ))

    fig.update_layout(
        title={'text': 'Gün ve Saat Kombinasyonlarına Göre Ziyaretçi Yoğunluğu', 'x': 0.5},
        width=700,
        height=400,
        xaxis_title="Saat",
        xaxis_tickangle=-45,
        
    )

    return fig




In [353]:
fig1 = daily_chart(data)
fig2 = weekly_chart(data)
fig3 = monthly_chart(data)
fig4 = daily_chart_gender(data)
fig5 = weekly_chart_gender(data)
fig6 = monthly_chart_gender(data)
fig7 = hourly_chart(data)
fig8 = hourly_gender_chart(data)
fig9 = monthly_day_distribution_count(data)
fig10 = monthly_day_distribution_average(data)
fig11 = quarter_pie_chart(data)
fig12 = gender_distribution_pie_chart(data)
fig13 = heatmap_hour_day(data)

encoded1 = image_to_ascii(fig1)
encoded2 = image_to_ascii(fig2)
encoded3 = image_to_ascii(fig3)
encoded4 = image_to_ascii(fig4)
encoded5 = image_to_ascii(fig5)
encoded6 = image_to_ascii(fig6)
encoded7 = image_to_ascii(fig7)
encoded8 = image_to_ascii(fig8)
encoded9 = image_to_ascii(fig9)
encoded10 = image_to_ascii(fig10)
encoded11 = image_to_ascii(fig11)
encoded12 = image_to_ascii(fig12)
encoded13 = image_to_ascii(fig13)

rapor_date = "01-01-2023 - 31-12-2023"

html_content = (
    """
<!DOCTYPE html>
<html>
<head>
    <title>PDF Example</title>
     <style>
        @page {
            margin: 0mm;
        }


        body {
            margin: 0;
            padding: 0px;
            width: 794px;
            height: 1122px;
            font-family: sans-serif;
        }

        .title {
            border-left: 4px solid #50b0fa;
            margin-left: 38px;
            margin-top: 50px;
            padding-left: 20px;
            font-size: 30px;
            font-weight: bold;

        }

        .wd-title {
            padding: 5px 0px;
            color: #124b68;

        }

        .company-title {
            font-size: 16px;
            color: #f32170;
            display: flex;
        }

       .chart-row {
            display: flex;
            justify-content: center;
            gap: 5px;
            margin-top: 20px;
        }


        .chart-container {
            box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px;
            
        }

         .chart-container img {
            width: 380px;
        }

        .rapor-date {
            right: 0;
            margin-left: auto;
            margin-right: 30px;
            color: #124b68;
        }

    </style>
    </style>
</head>

"""
    + f"""

  



    <body>

    <div class="title">
        <div class="wd-title">Watch Data Rapor</div>
        <div class="company-title">KELEBEK MOBİLYA <div class="rapor-date">{rapor_date}</div>  </div>
    </div>

    <div class="chart-row">
        <div class="chart-container">
            <img src="data:image/png;base64,{encoded1}" alt="Plotly Graph 1">
        </div>
        <div class="chart-container">
            <img src="data:image/png;base64,{encoded2}" alt="Plotly Graph 2">
        </div>

    </div>

    <div class="chart-row">
        <div class="chart-container">
            <img style="width: auto; height: auto;" src="data:image/png;base64,{encoded3}" alt="Plotly Graph 1">
        </div>
      

    </div>

    <div class="chart-row">
        <div class="chart-container">
            <img src="data:image/png;base64,{encoded4}" alt="Plotly Graph 1">
        </div>
        <div class="chart-container">
            <img src="data:image/png;base64,{encoded5}" alt="Plotly Graph 2">
        </div>

    </div>

    <div class="chart-row" style="padding-top: 50px">
        <div class="chart-container">
            <img style="width: auto; height: auto;" src="data:image/png;base64,{encoded6}" alt="Plotly Graph 1">
        </div>
    </div>

    <div class="chart-row" style="padding-top: 50px">
        <div class="chart-container">
            <img style="width: auto; height: auto;" src="data:image/png;base64,{encoded7}" alt="Plotly Graph 1">
        </div>
    </div>

    <div class="chart-row" style="padding-top: 50px">
        <div class="chart-container">
            <img style="width: auto; height: auto;" src="data:image/png;base64,{encoded8}" alt="Plotly Graph 1">
        </div>
    </div>

    <div class="chart-row" style="padding-top: 50px">
        <div class="chart-container">
            <img style="width: auto; height: auto;" src="data:image/png;base64,{encoded9}" alt="Plotly Graph 1">
        </div>
    </div>

    <div class="chart-row" style="padding-top: 50px">
        <div class="chart-container">
            <img style="width: auto; height: auto;" src="data:image/png;base64,{encoded10}" alt="Plotly Graph 1">
        </div>
    </div>


    <div class="chart-row">
        <div class="chart-container">
            <img src="data:image/png;base64,{encoded11}" alt="Plotly Graph 1">
        </div>
        <div class="chart-container">
            <img src="data:image/png;base64,{encoded12}" alt="Plotly Graph 2">
        </div>

    </div>

    <div class="chart-row" style="padding-top: 50px">
        <div class="chart-container">
            <img style="width: auto; height: auto;" src="data:image/png;base64,{encoded13}" alt="Plotly Graph 1">
        </div>
    </div>

</body>
</html>
"""
)

pdf_dosyasi = "output.pdf"

HTML(string=html_content).write_pdf(pdf_dosyasi)

print(f"PDF oluşturuldu: {pdf_dosyasi}")

PDF oluşturuldu: output.pdf
